# **Recurrent neural networks tutorial**
In today's tutorial we will discover how to develop Recurrent Neural Networks (RNNs) using [**Keras**](https://keras.io/) deep learning library to address time series forecasting problems.

# **Preliminary operations**
The following code downloads all the necessary material into the remote machine. At the end of the execution select the **File** tab to verify that everything has been correctly downloaded.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00492/Metro_Interstate_Traffic_Volume.csv.gz

!gzip -d Metro_Interstate_Traffic_Volume.csv.gz

# **Useful modules import**
First of all, it is necessary to import useful modules used during the tutorial.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import mean_squared_error
from ipywidgets import interact, fixed

# **Utility functions**
Execute the following code to define some utility functions used in the tutorial:
- **plot_history** draws in a graph the loss trend over epochs on both training and validation sets. Moreover, if provided, it draws in the same graph also the trend of the given metric;
- **plot_single_time_sequence** plots a selected time sequence with the corresponding actual and predicted values;
- **plot_time_series** plots the actual and the predicted time series;
- **prepare_multiple_targets** prepares the target sequence to have multiple steps to be used to forecast multiple future time steps.

In [ ]:
def plot_history(history,metric=None):
  fig, ax1 = plt.subplots(figsize=(10, 8))

  epoch_count=len(history.history['loss'])

  line1,=ax1.plot(range(1,epoch_count+1),history.history['loss'],label='train_loss',color='orange')
  ax1.plot(range(1,epoch_count+1),history.history['val_loss'],label='val_loss',color = line1.get_color(), linestyle = '--')
  ax1.set_xlim([1,epoch_count])
  ax1.set_ylim([0, max(max(history.history['loss']),max(history.history['val_loss']))])
  ax1.set_ylabel('loss',color = line1.get_color())
  ax1.tick_params(axis='y', labelcolor=line1.get_color())
  ax1.set_xlabel('Epochs')
  _=ax1.legend(loc='lower left')

  if (metric!=None):
    ax2 = ax1.twinx()
    line2,=ax2.plot(range(1,epoch_count+1),history.history[metric],label='train_'+metric)
    ax2.plot(range(1,epoch_count+1),history.history['val_'+metric],label='val_'+metric,color = line2.get_color(), linestyle = '--')
    ax2.set_ylim([0, max(max(history.history[metric]),max(history.history['val_'+metric]))])
    ax2.set_ylabel(metric,color=line2.get_color())
    ax2.tick_params(axis='y', labelcolor=line2.get_color())
    _=ax2.legend(loc='upper right')

def plot_single_time_sequence(y,y_pred,t_y_pred,timesteps,figsize=(20, 10)):
  plt.figure(figsize=figsize)

  y_range=range(t_y_pred,t_y_pred+timesteps)
  plt.plot(y_range,y[y_range], '.-',label='Time sequence')

  if y_pred.ndim==1:
    plt.scatter(t_y_pred+timesteps,y[t_y_pred+timesteps],marker='x',label='Actual values')
    plt.scatter(t_y_pred+timesteps,y_pred[t_y_pred],c='r',marker='o',label='Predicted values')
  else:
    y_pred_range=range(t_y_pred+timesteps,t_y_pred+timesteps+y_pred.shape[1])
    plt.scatter(y_pred_range,y[y_pred_range],marker='x',label='Actual values')
    plt.scatter(y_pred_range,y_pred[t_y_pred],c='r',marker='o',label='Predicted values')

  plt.grid(True)

  plt.xlabel('t', fontsize=14)
  plt.ylabel('x(t)', fontsize=14)

  plt.legend()

def plot_time_series(y,y_pred,t1_y,t2_y,timesteps,figsize=(20, 10)):
  plt.figure(figsize=figsize)

  y_range=range(t1_y,t2_y)
  plt.plot(y_range,y[y_range], '.-',label='Actual values')
  
  y_pred_range=range(t1_y+timesteps,t2_y)
  plt.plot(y_pred_range,y_pred[[t-timesteps for t in y_pred_range]], '.-r',label='Predicted values')
  
  plt.grid(True)
  
  plt.xlabel('t', fontsize=14)
  plt.ylabel('x(t)', fontsize=14)

  plt.legend(loc='upper left')

def prepare_multiple_targets(single_targets,multistep_count):
  single_target_row_count=single_targets.shape[0]
  
  multistep_targets=[]
  for i in range(single_target_row_count):
    target=np.zeros(multistep_count)
    for j in range(multistep_count):
      idx=i+j
      if idx<single_target_row_count:
        target[j]=single_targets[idx]
    multistep_targets.append(target)  

  return np.array(multistep_targets)

# **Dataset**
This tutorial uses the [Metro Interstate Traffic Volume Data Set](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume) manteined by the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php), a public repository containing hundreds of databases useful for the machine learning community.

The data set includes hourly Minneapolis-St Paul traffic volume for westbound I-94 from 2012-2018. It contains 48204 istances with 9 attributes:
- *holiday*: categorical US national holidays plus regional holiday;
- *temp*: Numeric average temperature in kelvin;
- *rain_1h*: numeric amount in mm of rain that occurred in the hour;
- *snow_1h*: numeric amount in mm of snow that occurred in the hour;
- *clouds_all*: numeric percentage of cloud cover;
- *weather_main*: categorical short textual description of the current weather;
- *weather_description*: Categorical longer textual description of the current weather;
- *date_time*: date and hour of the data collected in local CST time;
- *traffic_volume*: Numeric hourly I-94 ATR 301 reported westbound traffic volume.

The dataset is stored in a CSV file and can be easily loaded in memory using [**pandas**](https://pandas.pydata.org/), a software library for data manipulation and analysis.

In [ ]:
dataframe = pd.read_csv('Metro_Interstate_Traffic_Volume.csv')

The variable *dataframe* is an instance of the pandas class [**DataFrame**](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html), a 2-dimensional labeled data structure with columns of potentially different types.

## **Visualization**
The first *row_count* selected rows can be shown by executing the following code.

In [ ]:
row_count=5

dataframe.head(row_count)

## **Statistics**
The [**info**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) method can be used to print a brief summary of a **DataFrame** including the index and the type of each column, the non-null values and the memory usage.

In [ ]:
dataframe.info()

Note that, there are no missing values in the dataset.

To show the overall statistics of the dataset can be used the method [**describe**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html).

In [ ]:
dataframe.describe().transpose()

From the statistics it is clear how each feature covers a very different range.

The method [**hist**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html) draws a histogram for each column in the **DataFrame**.

In [ ]:
dataframe.hist(bins=50, figsize=(20,15))
plt.show()

## **Data preparation**
Most machine learning algorithms require data to be formatted in a specific way, so datasets generally require some amount of preparation before they can yield useful insights. Some datasets have values that are missing, invalid, or otherwise difficult for an algorithm to process.

### **Encode cyclical data**
Traffic volume is strongly related to the time of the day (e.g., 9 A.M. or 10 P.M.), the day of the week (e.g., Monday or Saturday) and the time of the year (e.g., January or August).

The following code extracts the hours from the *date_time* column and plots them in a graph.

In [ ]:
hour=dataframe['date_time'].astype('datetime64').dt.hour
hour[:160].plot()

The graph report the hourly data for a week: a cycle between 0 and 23 that repeats 7 times presenting a **jump discontinuity** at the end of each day, when the hour value goes from  23  to  00.

Presenting cyclical data to a machine learning algorithm is a problem. For instance, it would consider the difference between 23 and 00 greater than that between 22 and 23.

A common method for encoding cyclical data is to transform the data into two dimensions using a sine and cosine transformation.

The hour sine and cosine values are computed and plotted by executing the following code.

In [ ]:
hour_sin = np.sin(2 * np.pi * hour/23.0)
hour_cos = np.cos(2 * np.pi * hour/23.0)

plt.figure(figsize=(5, 5))
plt.xlabel('hour_sin')
plt.ylabel('hour_cos')
plt.scatter(hour_sin,hour_cos)

As expected, the hour information are encoded as a cycle.

The two new features (*hour_sin* and *hour_cos*) can be inserted in the **DataFrame** using the [**insert**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.insert.html) method.

In [ ]:
prepared_dataframe=dataframe.copy()

prepared_dataframe.insert(len(prepared_dataframe.columns)-1,'hour_sin', hour_sin)
prepared_dataframe.insert(len(prepared_dataframe.columns)-1,'hour_cos', hour_cos)

prepared_dataframe.head(row_count)

The same thing can be done with the day of the week and the month by executing the following cell.

In [ ]:
day_week=dataframe['date_time'].astype('datetime64').dt.dayofweek
day_week_sin = np.sin(2 * np.pi * day_week/7.0)
day_week_cos = np.cos(2 * np.pi * day_week/7.0)

month=dataframe['date_time'].astype('datetime64').dt.month
month_sin = np.sin(2 * np.pi * month/12.0)
month_cos = np.cos(2 * np.pi * month/12.0)

prepared_dataframe.insert(len(prepared_dataframe.columns)-1,'day_week_sin', day_week_sin)
prepared_dataframe.insert(len(prepared_dataframe.columns)-1,'day_week_cos', day_week_cos)

prepared_dataframe.insert(len(prepared_dataframe.columns)-1,'month_sin', month_sin)
prepared_dataframe.insert(len(prepared_dataframe.columns)-1,'month_cos', month_cos)

prepared_dataframe.head(row_count)

### **Remove unuseful columns**
The *date_time* column contains no useful information. It can be removed from the dataset using the [**drop**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) method.

In [ ]:
prepared_dataframe=prepared_dataframe.drop(['date_time'],axis=1)
prepared_dataframe.head(row_count)

### **Convert categorical data**
The *holiday*, *weather_main* and *weather_description* columns are categorical, not numeric. Their conversion into numeric format can be done in two ways: 
- *label encoding*, converting each category to a number;
- *one hot encoding*, converting each category value into a new column and assigns a 1 or 0 (True/False) value to the column.

**Label encoding**

First of all, if the column type is *object* and not *category*, the [**astype**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) method can be used to convert a column to a category.

Then the encoded values can be assigned to the corresponding column using the cat.codes accessor.

In [ ]:
label_enc_dataframe=prepared_dataframe.copy()

label_enc_dataframe['holiday'] = prepared_dataframe['holiday'].astype('category')
label_enc_dataframe['weather_main'] = prepared_dataframe['weather_main'].astype('category')
label_enc_dataframe['weather_description'] = prepared_dataframe['weather_description'].astype('category')

label_enc_dataframe['holiday'] = label_enc_dataframe['holiday'].cat.codes
label_enc_dataframe['weather_main'] = label_enc_dataframe['weather_main'].cat.codes
label_enc_dataframe['weather_description'] = label_enc_dataframe['weather_description'].cat.codes
label_enc_dataframe.head(row_count)

Label encoding has the advantage that it is straightforward but it has the disadvantage that the numeric values can be “misinterpreted” by the algorithms. For example, the value of 0 is obviously less than the value of 4 but does that really correspond to reality (e.g., *weather_description*)?

**One hot encoding**

Pandas supports this feature using the [**get_dummies**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function.

In [ ]:
one_hot_enc_dataframe=pd.get_dummies(prepared_dataframe, columns=['holiday', 'weather_main','weather_description'], prefix=['holiday', 'weather_main','weather_description'])
one_hot_enc_dataframe.head(row_count)

One hot encoding has the benefit of not weighting a value improperly but does have the downside of adding more columns to the data set (it depends by the number of categories in a column).

**What is the best solution?**

It depends on the specific dataset used.

In this tutorial, because *holiday*, *weather_main* and *weather_description* columns contain several categories (12, 11 and 38, respectively), it is better to use the *label encoding* solution.

In [ ]:
used_dataframe=label_enc_dataframe
#used_dataframe=one_hot_enc_dataframe

## **Split data into training, validation and test sets**
In order to avoid overfitting during training and to evaluate the generalization capabilites of the models, it is necessary to divide the data into three disjoined datasets: training, validation and test sets.

The *train_size_perc* and *val_size_perc* parameters represent the percentage of patterns to include in the training and validation sets, respectively. The remaining patterns will be used to create the test set.

<u>Because it is necessary to maintain the data ordered, no mixing routine is used.</u>

The Numpy representation of the **DataFrame** can be obtained using the [**values**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.values.html) property.

In [ ]:
train_size_perc=0.5
val_size_perc=0.25

train_size=int(train_size_perc*len(used_dataframe))
val_size=int(val_size_perc*len(used_dataframe))

train_data=used_dataframe.values[:train_size, :]
val_data=used_dataframe.values[train_size:(train_size+val_size), :]
test_data=used_dataframe.values[(train_size+val_size):len(used_dataframe), :]

print('Train data shape: ',train_data.shape)
print('Validation data shape: ',val_data.shape)
print('Test data shape: ',test_data.shape)

## **Data normalization**
It is good practice to normalize features that use different scales and ranges.

Scikit-learn library provides the class [**StandardScaler**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to normalize features by removing the mean and scaling to unit variance.

<u>Note that, the target values (the *traffic_volume* column) are copied before the normalization to use them unscaled during the performance evaluation.</u>

In [ ]:
train_y=np.copy(train_data[:,-1])
val_y=np.copy(val_data[:,-1])
test_y=np.copy(test_data[:,-1])

scaler_x = StandardScaler()
scaler_x.fit(train_data[:,:-1])
train_data[:,:-1]=scaler_x.transform(train_data[:,:-1])
val_data[:,:-1]=scaler_x.transform(val_data[:,:-1])
test_data[:,:-1]=scaler_x.transform(test_data[:,:-1])

scaler_y = StandardScaler()
scaler_y.fit(train_data[:,-1].reshape(-1,1))
train_data[:,-1]=scaler_y.transform(train_data[:,-1].reshape(-1,1)).squeeze()
val_data[:,-1]=scaler_y.transform(val_data[:,-1].reshape(-1,1)).squeeze()
test_data[:,-1]=scaler_y.transform(test_data[:,-1].reshape(-1,1)).squeeze()

## **Time series data preparation**
Time series data must be transformed into a structure of samples with input and output components before it can be used to fit a supervised learning model.

Keras provides the [**TimeseriesGenerator**](https://keras.io/api/preprocessing/timeseries/#timeseriesgenerator-class) class to automatically transform time series data into samples, ready to train deep learning models given:
- the input features (*data*);
- the actual outputs (*targets*);
- the length (number of timesteps, observations or rows) of the generated sequences (*length*);
- the number of time sequence samples in each batch (*batch_size*).

In [ ]:
timesteps=12
batch_size=256

train_data_gen = TimeseriesGenerator(data=train_data, targets=train_data[:,-1],
                                     length=timesteps, batch_size=batch_size)
val_data_gen = TimeseriesGenerator(data=val_data, targets=val_data[:,-1],
                                  length=timesteps, batch_size=batch_size)
test_data_gen= TimeseriesGenerator(data=test_data, targets=test_data[:,-1],
                                    length=timesteps, batch_size=batch_size)

The batch number of a **TimeseriesGenerator** instance can be obtained using the **len** function.

In [ ]:
print('Number of train batches: ',len(train_data_gen))
print('Number of validation batches: ',len(val_data_gen))
print('Number of test batches: ',len(test_data_gen))

Batches of a **TimeseriesGenerator** instance can be accessed by referring to its index number. 

In [ ]:
x,y = train_data_gen[0]

print('Time sequences feature batch shape: ',x.shape)
print('Time sequences target batch shape: ',y.shape)

print('First feature time sequence:\n',x[0])
print('First target value: ',y[0])

Once a **TimeseriesGenerator** instance has been defined, it can be used to train a neural network model as a data generator.

# **Simple RNN**
In this section a simple *many-to-one* RNN will be created.

![alt text](https://biolab.csr.unibo.it/ferrara/Courses/DL/Tutorials/RNN/ManyToOne_RNN.png)

## **Model definition**
The following function creates a simple RNN model given:
- the number of timesteps in each input sequence (*timesteps*);
- the number of features in each timestep (*feature_count*).

The model returns a single target value given an entire sequence as input (*many-to-one*).

<u>Note that, the number of timesteps in each input sequence (*timesteps*) is set in advance only because it improves performance during training by creating tensors of fixed shapes. A *None* value can be used to admit variable-length input sequences.</u>

In Keras, a sequential is a stack of layers where each layer has exactly one input and one output. It can be created by passing a list of layers to the  constructor [**keras.Sequential**](https://keras.io/guides/sequential_model/).

[**Keras layers API**](https://keras.io/api/layers/) offers a wide range of built-in layers ready for use, including:
- [**Input**](https://keras.io/api/layers/core_layers/input/) - the input of the model. Note that, you can also omit the **Input** layer. In that case the model doesn't have any weights until the first call to a training/evaluation method (since it is not yet built);
- [**SimpleRNN**](https://keras.io/api/layers/recurrent_layers/simple_rnn/) - a fully-connected RNN where the output is to be fed back to input.

In [ ]:
def build_simple_rnn(timesteps,feature_count):
  model = keras.Sequential(
      [
        layers.Input(shape=(timesteps,feature_count)),
        layers.SimpleRNN(1)
      ]
    )

  return model

## **Model creation**
The following code creates a simple RNN model by calling the **build_simple_rnn** function defined above.

In [ ]:
simple_rnn=build_simple_rnn(timesteps,train_data.shape[1])

## **Model visualization**
A string summary of the network can be printed by executing the following code.

In [ ]:
simple_rnn.summary()

Alternatively, a plot of the neural network graph can be visualized.

In [ ]:
keras.utils.plot_model(simple_rnn,show_shapes=True, show_layer_names=False)

## **Model compilation**
The compilation is the final step in configuring the model for training. 

The following code use the [**compile**](https://keras.io/api/models/model_training_apis/#compile-method) method to compile the model.
The important arguments are:
- the optimization algorithm (*optimizer*);
- the loss function (*loss*);
- the metrics used to evaluate the performance of the model (*metrics*).

The most common [optimization algorithms](https://keras.io/api/optimizers/#available-optimizers), [loss functions](https://keras.io/api/losses/#available-losses) and [metrics](https://keras.io/api/metrics/#available-metrics) are already available in Keras. You can either pass them to **compile** as an instance or by the corresponding string identifier. In the latter case, the default parameters will be used.

In [ ]:
simple_rnn.compile(loss='mse', optimizer='adam',metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

## **Training**
Now we are ready to train our model by calling the [**fit**](https://keras.io/api/models/model_training_apis/#fit-method) method.

It trains the model for a fixed number of epochs (*epoch_count*) using the training and the validation generators (*train_data_gen* and *val_data_gen*).

Break training when a metric or the loss has stopped improving on the validation set, helps to avoid overfitting.

For this purpose, Keras provides a class called [**EarlyStopping**](https://keras.io/api/callbacks/early_stopping/). Important class parameters are:
- *monitor* - the name of the metric or the loss to be observed; 
- *patience* - the number of epochs with no improvement after which training will be stopped;
- *restore_best_weights* - whether to restore model weights from the epoch with the best value of the monitored quantity.

Once created an instance of the **EarlyStopping** class, it can be passed to the **fit** method in the *callbacks* parameter.

In [ ]:
epoch_count = 100
patience=5

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

history = simple_rnn.fit(train_data_gen,validation_data=val_data_gen,epochs=epoch_count,callbacks=[early_stop])

### **Visualize the training process**
We can learn a lot about our model by observing the graph of its performance over time during training.

The **fit** method returns an object (*history*) containing loss and metrics values at successive epochs for both training and validation sets.

The following code calls the **plot_history** function defined above to draw in a graph the loss and RMSE trend over epochs on both training and validation sets.

In [ ]:
plot_history(history,'rmse')

## **Performance evaluation**
The following code calls the [**predict**](https://keras.io/api/models/model_training_apis/#predict-method) method to generate the predictions of the training, validation and test sets.

In [ ]:
scaled_train_y_pred=simple_rnn.predict(train_data_gen)
scaled_val_y_pred=simple_rnn.predict(val_data_gen)
scaled_test_y_pred=simple_rnn.predict(test_data_gen)

print('Train predictions shape: ',scaled_train_y_pred.shape)
print('Validation predictions shape: ',scaled_val_y_pred.shape)
print('Test predictions shape: ',scaled_test_y_pred.shape)

The returned predictions are scaled because the target values have been normalized before the training process. To obtain the unscaled predictions, it is sufficient to call the [**inverse_transform**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.inverse_transform) method of the **StandardScaler** instance with the scaled predictions as input.  

In [ ]:
train_y_pred=scaler_y.inverse_transform(scaled_train_y_pred)
val_y_pred=scaler_y.inverse_transform(scaled_val_y_pred)
test_y_pred=scaler_y.inverse_transform(scaled_test_y_pred)

### **RMSE**
The regression accuracy can be measured using the RMSE.

Scikit-learn library provides the function [**mean_squared_error**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) to compute MSE and RMSE metrics.

If the *squared* parameter is set to False, the function returns the RMSE value.

In [ ]:
rmse_train = mean_squared_error(train_y[timesteps:],train_y_pred,squared=False)
rmse_val = mean_squared_error(val_y[timesteps:],val_y_pred,squared=False)
rmse_test = mean_squared_error(test_y[timesteps:],test_y_pred,squared=False)

print('RMSE')
print('Train: {:.2f}'.format(rmse_train))
print('Validation: {:.2f}'.format(rmse_val))
print('Test: {:.2f}'.format(rmse_test))

### **Best and worst predictions**
To visualize test predictions sorted by RMSE, its value needs to be computed for each test sequence and then sorted.

In [ ]:
rmse_test_sequences=np.sqrt(mean_squared_error(np.expand_dims(test_y[timesteps:],axis=1).transpose(),
                                                test_y_pred.transpose(),multioutput='raw_values'))
rmse_test_sequences_sorted_indices=np.argsort(rmse_test_sequences)

The following code shows the test predictions sorted by RMSE. The specific prediction can be selected by moving the slider from the best (left) to the worst (right) one.

In [ ]:
@interact(y=fixed(test_y),
         y_pred=fixed(test_y_pred),
         rmse_values=fixed(rmse_test_sequences),
         rmse_sorted_indices=fixed(rmse_test_sequences_sorted_indices),
         selected_index=widgets.IntSlider(min=0, max=rmse_test_sequences_sorted_indices.shape[0]-1, step=1, value=0,continuous_update=False))
def interactive_plot(y,y_pred,rmse_values,rmse_sorted_indices,selected_index):
  print('RMSE: {:.2f}'.format(rmse_values[rmse_sorted_indices[selected_index]]))
  plot_single_time_sequence(y,y_pred,rmse_sorted_indices[selected_index],timesteps,(10,5))

### **Plot time sequence predictions**
To better evaluate the model performance on the test set, it is useful to plot the actual and the predicted sequences.

The following code plots a portion or the entire time series with the corresponding forecast given the initial (*t1*) and the final (*t2*) times.

In [ ]:
t1=0    #0 to visualize the entire time series
t2=200  #test_y.shape[0] to visualize the entire time series

plot_time_series(test_y,test_y_pred,t1,t2,timesteps)

# **Deep RNN**
In this section a deep RNN will be created.

![alt text](https://biolab.csr.unibo.it/ferrara/Courses/DL/Tutorials/RNN/DeepRNN.png)

## **Model definition**
The following function creates a deep DNN model given:
- the number of timesteps in each input sequence (*timesteps*);
- the number of features in each timestep (*feature_count*);
- the number of units for each RNN layer (*unit_count_per_rnn_layer*).

The model returns a single target value given an entire sequence as input (*many-to-one*).

<u>Note that, the number of timesteps in each input sequence (*timesteps*) is set in advance only because it improves performance during training by creating tensors of fixed shapes. A *None* value can be used to admit variable-length input sequences.</u>

The *return_sequences* parameter of the **SimpleRNN** layer serves to return the full output time sequence (True), or only the last output (False).

In [ ]:
def build_deep_rnn(timesteps,feature_count,unit_count_per_rnn_layer=[128,128]):
  model = keras.Sequential()
  model.add(layers.Input(shape=(timesteps,feature_count)))

  for i in range(len(unit_count_per_rnn_layer)):
    model.add(layers.SimpleRNN(unit_count_per_rnn_layer[i],return_sequences=True if i<(len(unit_count_per_rnn_layer)-1) else False))

  if unit_count_per_rnn_layer[-1]>1:
    model.add(layers.Dense(1))

  return model

## **Model creation**
The following code creates a deep RNN model by calling the **build_deep_rnn** function defined above.

In [ ]:
deep_rnn=build_deep_rnn(timesteps,train_data.shape[1])

## **Model visualization**
A string summary of the network can be printed by executing the following code.

In [ ]:
deep_rnn.summary()

Alternatively, a plot of the neural network graph can be visualized.

In [ ]:
keras.utils.plot_model(deep_rnn,show_shapes=True, show_layer_names=False)

## **Model compilation**
The following code compiles the model as already done before.

In [ ]:
deep_rnn.compile(loss='mse', optimizer='adam',metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

## **Training**
Now we are ready to train our model by calling the **fit** method.

In [ ]:
epoch_count = 100
patience=5

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

history = deep_rnn.fit(train_data_gen,validation_data=val_data_gen,epochs=epoch_count,callbacks=[early_stop])

### **Visualize the training process**
The following code calls the **plot_history** function defined above to draw in a graph the loss and RMSE trend over epochs on both training and validation sets.

In [ ]:
plot_history(history,'rmse')

## **Performance evaluation**
The following code calls the **predict** method to generate the predictions of the training, validation and test sets.

In [ ]:
scaled_train_y_pred=deep_rnn.predict(train_data_gen)
scaled_val_y_pred=deep_rnn.predict(val_data_gen)
scaled_test_y_pred=deep_rnn.predict(test_data_gen)

print('Train predictions shape: ',scaled_train_y_pred.shape)
print('Validation predictions shape: ',scaled_val_y_pred.shape)
print('Test predictions shape: ',scaled_test_y_pred.shape)

The returned predictions are scaled because the target values have been normalized before the training process. To obtain the unscaled predictions, it is sufficient to call the **inverse_transform** method of the **StandardScaler** instance with the scaled predictions as input. 

In [ ]:
train_y_pred=scaler_y.inverse_transform(scaled_train_y_pred)
val_y_pred=scaler_y.inverse_transform(scaled_val_y_pred)
test_y_pred=scaler_y.inverse_transform(scaled_test_y_pred)

### **RMSE**
The regression accuracy can be measured using the RMSE.

In [ ]:
rmse_train = mean_squared_error(train_y[timesteps:],train_y_pred,squared=False)
rmse_val = mean_squared_error(val_y[timesteps:],val_y_pred,squared=False)
rmse_test = mean_squared_error(test_y[timesteps:],test_y_pred,squared=False)

print('RMSE')
print('Train: {:.2f}'.format(rmse_train))
print('Validation: {:.2f}'.format(rmse_val))
print('Test: {:.2f}'.format(rmse_test))

### **Best and worst predictions**
To visualize test predictions sorted by RMSE, its value needs to be computed for each test sequence and then sorted.

In [ ]:
rmse_test_sequences=np.sqrt(mean_squared_error(np.expand_dims(test_y[timesteps:],axis=1).transpose(),
                                                test_y_pred.transpose(),multioutput='raw_values'))
rmse_test_sequences_sorted_indices=np.argsort(rmse_test_sequences)

The following code shows the test predictions sorted by RMSE. The specific prediction can be selected by moving the slider from the best (left) to the worst (right) one.

In [ ]:
@interact(y=fixed(test_y),
         y_pred=fixed(test_y_pred),
         rmse_values=fixed(rmse_test_sequences),
         rmse_sorted_indices=fixed(rmse_test_sequences_sorted_indices),
         selected_index=widgets.IntSlider(min=0, max=rmse_test_sequences_sorted_indices.shape[0]-1, step=1, value=0,continuous_update=False))
def interactive_plot(y,y_pred,rmse_values,rmse_sorted_indices,selected_index):
  print('RMSE: {:.2f}'.format(rmse_values[rmse_sorted_indices[selected_index]]))
  plot_single_time_sequence(y,y_pred,rmse_sorted_indices[selected_index],timesteps,(10,5))

### **Plot time sequence predictions**
The following code plots a portion or the entire time series with the corresponding forecast given the initial (*t1*) and the final (*t2*) times.

In [ ]:
t1=0    #0 to visualize the entire time series
t2=200  #test_y.shape[0] to visualize the entire time series

plot_time_series(test_y,test_y_pred,t1,t2,timesteps)

# **Exercise 1**
Improve the performance of the deep RNN model. It is recommended to evaluate the following hyperparameters (listed in priority order):
1. the number of units for each RNN layer (*unit_count_per_rnn_layer*);
2. the number of timesteps in each input sequence (*timesteps*);
3. the mini-batch size (*batch_size*);
4. the number of training epochs (*epoch_count*).

Note that, points 2 and 3 require to recreate the **TimeseriesGenerator** instances.

# **Multi-step Forecasts**
Neural networks can learn to map an input pattern to an output pattern of more than one feature. This can be used in time series forecasting to directly forecast multiple future time steps.

This can be achieved either by directly outputting a vector from the model, by specifying the desired number of outputs as the number of nodes in the output layer.

A limitation of the **TimeseriesGenerator** class is that it does not directly support multi-step outputs. Specifically, it will not create the multiple steps that may be required in the target sequence.

Nevertheless, if we prepare the target sequence to have multiple steps, the **TimeseriesGenerator** class will use them as the output portion of each sample.

The following code prepare the training, validation and test target sequences to have *multistep_count* steps.

In [ ]:
multistep_count=24

multistep_train_targets=prepare_multiple_targets(train_data[:,-1],multistep_count)
multistep_val_targets=prepare_multiple_targets(val_data[:,-1],multistep_count)
multistep_test_targets=prepare_multiple_targets(test_data[:,-1],multistep_count)

print('Multistep train targets shape: ',multistep_train_targets.shape)
print('Multistep validation targets shape: ',multistep_val_targets.shape)
print('Multistep test targets shape: ',multistep_test_targets.shape)

## **Time series data preparation**
Now the time series data can be transformed into a structure of samples with input and output components using the **TimeseriesGenerator** class.

In [ ]:
timesteps=12
batch_size=256

multistep_train_data_gen = TimeseriesGenerator(data=train_data[:-multistep_count+1,:],
                                               targets=multistep_train_targets[:-multistep_count+1],
                                               length=timesteps, batch_size=batch_size)
multistep_val_data_gen = TimeseriesGenerator(data=val_data[:-multistep_count+1,:],
                                             targets=multistep_val_targets[:-multistep_count+1],
                                             length=timesteps, batch_size=batch_size)
multistep_test_data_gen= TimeseriesGenerator(data=test_data[:-multistep_count+1,:],
                                             targets=multistep_test_targets[:-multistep_count+1],
                                             length=timesteps, batch_size=batch_size)

The batch number of a **TimeseriesGenerator** instance can be obtained using the **len** function.

In [ ]:
print('Number of train batches: ',len(multistep_train_data_gen))
print('Number of validation batches: ',len(multistep_val_data_gen))
print('Number of test batches: ',len(multistep_test_data_gen))

Batches of a **TimeseriesGenerator** instance can be accessed by referring to its index number. 

In [ ]:
x,y = multistep_train_data_gen[0]

print('Time sequences feature batch shape: ',x.shape)
print('Time sequences target batch shape: ',y.shape)

print('First feature time sequence:\n',x[0])
print('First target value:\n',y[0])

## **Model definition**
The following function creates a deep DNN model for multi-step forecasts given:
- the number of timesteps in each input sequence (*timesteps*);
- the number of features in each timestep (*feature_count*);
- the number of multi-steps in each target sequence (*target_count*);
- the number of units for each RNN layer (*unit_count_per_rnn_layer*).

The model returns a multi-step target value given an entire sequence as input.

<u>Note that, the number of timesteps in each input sequence (*timesteps*) is set in advance only because it improves performance during training by creating tensors of fixed shapes. A *None* value can be used to admit variable-length input sequences.</u>

The *return_sequences* parameter of the **SimpleRNN** layer serves to return the full output time sequence (True), or only the last output (False).

In [ ]:
def build_multistep_deep_rnn(timesteps,feature_count,target_count,unit_count_per_rnn_layer=[128,128]):
  model = keras.Sequential()
  model.add(layers.Input(shape=(timesteps,feature_count)))

  for i in range(len(unit_count_per_rnn_layer)):
    model.add(layers.SimpleRNN(unit_count_per_rnn_layer[i],return_sequences=True if i<(len(unit_count_per_rnn_layer)-1) else False))

  if unit_count_per_rnn_layer[-1]!=target_count:
    model.add(layers.Dense(target_count))

  return model

## **Model creation**
The following code creates a deep RNN model for multi-step forecasts by calling the **build_multistep_deep_rnn** function defined above.

In [ ]:
multistep_deep_rnn=build_multistep_deep_rnn(timesteps,train_data.shape[1],multistep_count)

## **Model visualization**
A string summary of the network can be printed by executing the following code.

In [ ]:
multistep_deep_rnn.summary()

Alternatively, a plot of the neural network graph can be visualized.

In [ ]:
keras.utils.plot_model(multistep_deep_rnn,show_shapes=True, show_layer_names=False)

## **Model compilation**
The following code compiles the model as already done before.

In [ ]:
multistep_deep_rnn.compile(loss='mse', optimizer='adam',metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

## **Training**
Now we are ready to train our model by calling the **fit** method.

In [ ]:
epoch_count = 100
patience=5

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

history = multistep_deep_rnn.fit(multistep_train_data_gen,validation_data=multistep_val_data_gen,epochs=epoch_count,callbacks=[early_stop])

### **Visualize the training process**
The following code calls the **plot_history** function defined above to draw in a graph the loss and RMSE trend over epochs on both training and validation sets.

In [ ]:
plot_history(history,'rmse')

## **Performance evaluation**
The following code calls the **predict** method to generate the predictions of the training, validation and test sets.

In [ ]:
multistep_scaled_train_y_pred=multistep_deep_rnn.predict(multistep_train_data_gen)
multistep_scaled_val_y_pred=multistep_deep_rnn.predict(multistep_val_data_gen)
multistep_scaled_test_y_pred=multistep_deep_rnn.predict(multistep_test_data_gen)

print('Multi-step train predictions shape: ',multistep_scaled_train_y_pred.shape)
print('Multi-step validation predictions shape: ',multistep_scaled_val_y_pred.shape)
print('Multi-step test predictions shape: ',multistep_scaled_test_y_pred.shape)

The returned predictions are scaled because the target values have been normalized before the training process. To obtain the unscaled predictions, it is sufficient to call the **inverse_transform** method of the **StandardScaler** instance with the scaled predictions as input. 

In [ ]:
multistep_train_y_pred=scaler_y.inverse_transform(multistep_scaled_train_y_pred)
multistep_val_y_pred=scaler_y.inverse_transform(multistep_scaled_val_y_pred)
multistep_test_y_pred=scaler_y.inverse_transform(multistep_scaled_test_y_pred)

Moreover, it is necessary to prepare the training, validation and test targets to have *multistep_count* steps.

In [ ]:
multistep_train_y=prepare_multiple_targets(train_y,multistep_count)
multistep_val_y=prepare_multiple_targets(val_y,multistep_count)
multistep_test_y=prepare_multiple_targets(test_y,multistep_count)

### **RMSE**
The regression accuracy can be measured using the RMSE.

In [ ]:
multistep_rmse_train = mean_squared_error(multistep_train_y[timesteps:-multistep_count+1],multistep_train_y_pred,squared=False)
multistep_rmse_val = mean_squared_error(multistep_val_y[timesteps:-multistep_count+1],multistep_val_y_pred,squared=False)
multistep_rmse_test = mean_squared_error(multistep_test_y[timesteps:-multistep_count+1],multistep_test_y_pred,squared=False)

print('RMSE')
print('Multi-step train: {:.2f}'.format(multistep_rmse_train))
print('Multi-step validation: {:.2f}'.format(multistep_rmse_val))
print('Multi-step test: {:.2f}'.format(multistep_rmse_test))

### **Best and worst predictions**
To visualize test predictions sorted by RMSE, its value needs to be computed for each test sequence and then sorted.

In [ ]:
rmse_test_sequences=np.sqrt(mean_squared_error(multistep_test_y[timesteps:-multistep_count+1].transpose(),
                                                multistep_test_y_pred.transpose(),multioutput='raw_values'))
rmse_test_sequences_sorted_indices=np.argsort(rmse_test_sequences)

The following code shows the test predictions sorted by RMSE. The specific prediction can be selected by moving the slider from the best (left) to the worst (right) one.

In [ ]:
@interact(y=fixed(test_y),
         y_pred=fixed(multistep_test_y_pred),
         rmse_values=fixed(rmse_test_sequences),
         rmse_sorted_indices=fixed(rmse_test_sequences_sorted_indices),
         selected_index=widgets.IntSlider(min=0, max=rmse_test_sequences_sorted_indices.shape[0]-1, step=1, value=0,continuous_update=False))
def interactive_plot(y,y_pred,rmse_values,rmse_sorted_indices,selected_index):
  print('RMSE: {:.2f}'.format(rmse_values[rmse_sorted_indices[selected_index]]))
  plot_single_time_sequence(y,y_pred,rmse_sorted_indices[selected_index],timesteps,(10,5))

# **Exercise 2**
Improve the performance of the deep RNN model on the multi-step forecast problem. It is recommended to evaluate the following hyperparameters (listed in priority order):
1. the number of units for each RNN layer (*unit_count_per_rnn_layer*);
2. the number of timesteps in each input sequence (*timesteps*);
3. the mini-batch size (*batch_size*);
4. the number of training epochs (*epoch_count*).

Note that, points 2 and 3 require to recreate the **TimeseriesGenerator** instances.

# **Exercise 3**
Keras provides specific layers to implement *Long Short-Term Memory* ([**LSTM**](https://keras.io/api/layers/recurrent_layers/lstm/)) and *Gated Recurrent Units* ([**GRU**](https://keras.io/api/layers/recurrent_layers/gru/)) networks.

Evaluate the performance of LSTM and GRU networks on both single- and multi-step forecast problem. 

Functions **build_deep_rnn** and **build_multistep_deep_rnn** defined above can be used as starting point by replacing the **SimpleRNN** layers with **LSTM** or **GRU** layers.

# **Exercise 4**
Solve another time series forecasting problem chosen from the following list:
- [Beijing Multi-Site Air-Quality Data Data Set](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data);
- [DJIA 30 Stock Time Series](https://www.kaggle.com/szrlee/stock-time-series-20050101-to-20171231);
- [Population Time Series Data](https://www.kaggle.com/census/population-time-series-data);
- [House Property Sales Time Series](https://www.kaggle.com/htagholdings/property-sales);
- [House Hold Energy Data - Time Series](https://www.kaggle.com/jaganadhg/house-hold-energy-data).
